In [1]:
import numpy as np
import sys, os
import time
import glob
from registration_3d import *
from cartesian import *
from collections import namedtuple
from operator import itemgetter
from pprint import pformat

In [2]:
def read_mesh( data_dir , filename ):
# get the coordinate of points
# get the vertices of each triangle
    mesh = glob.glob( data_dir + filename )
    mesh_file = open( mesh[0] , "r")
    mesh_lines = mesh_file.read().splitlines()
    mesh_data = []
    
    for num in range(len(mesh_lines)):
        mesh_data.append( mesh_lines[num].split(' ') )
        for i in range(len( mesh_data[-1])):
            mesh_data[-1][i] = mesh_data[-1][i].strip() # to remove space and tabs
    Np = int(mesh_data[0][0])
    mesh_data = mesh_data[1:]
    
    mesh_points = np.asarray(mesh_data[0:Np]).astype(float)
    mesh_data = mesh_data[Np:]
    
    Ntriangles = int(mesh_data[0][0])
    mesh_data = mesh_data[1:]
    mesh_vertices = np.asarray(mesh_data[0:Ntriangles]).astype(int)
    mesh_data = mesh_data[Ntriangles:]
    
    return Np,mesh_points,Ntriangles,mesh_vertices

In [3]:
def read_body( data_dir , filename ):

# get the markers' positions and tip of a body
    body = glob.glob( data_dir + filename )
    body_file = open( body[0] , "r")
    body_lines = body_file.read().splitlines()
    body_data = []
    
    for num in range(len(body_lines)):
        body_data.append( body_lines[num].split() )
        for i in range(len( body_data[-1])):
            body_data[-1][i] = body_data[-1][i].strip() # to remove space and tabs
    Np = int(body_data[0][0])
    body_data = body_data[1:]
    
    body_points = np.asarray(body_data[0:Np]).astype(float)
    body_data = body_data[Np:]
    
    tip = body_data[0]
    tip = np.array(tip).astype(float)
    tip = tip.reshape(1,-1)
    return Np,body_points,tip


In [4]:
def read_sample( data_dir , filename , N_Ap , N_Bp ):
# get the markers' positions and tip of a body
# N_Ap is the number of marker in A
# N_Bp is the number of marker in B
    sample = glob.glob( data_dir + filename )
    sample_file = open( sample[0] , "r")
    sample_lines = sample_file.read().splitlines()
    sample_data = []
    
    for num in range(len(sample_lines)):
        sample_data.append( sample_lines[num].split(',') )
        for i in range(len( sample_data[-1])):
            sample_data[-1][i] = sample_data[-1][i].strip() # to remove space and tabs
    Np = int(sample_data[0][0])
    N_sample = int(sample_data[0][1])
    N_Dp = Np - N_Ap - N_Bp
    sample_data = sample_data[1:]
    sample_data = np.array(sample_data).astype(float)
    A_frames=[]
    B_frames=[]
    
    for i in range(N_sample):
        A_sample_points = np.asarray( sample_data[ i*Np : i*Np + N_Ap ] ).astype( float )
        A_frames.append( A_sample_points )
        
        B_sample_points = np.asarray( sample_data[ i*Np + N_Ap :  i*Np + N_Ap + N_Bp] ).astype( float )
        B_frames.append( B_sample_points )

    A_frames = np.array(A_frames).astype(float)
    B_frames = np.array(B_frames).astype(float)
        
    return A_frames , B_frames

In [5]:
def get_d_k(A_frames, A_points, A_tip, B_frames, B_points, B_tip):
    
    d_k = []
    for i in range(len(A_frames)):
        F_AK = registration_3d(A_points , A_frames[i])
        F_BK = registration_3d(B_points , B_frames[i])
        # F_AK is the transformation from A tool frame to optical frame
        # F_AK is the transformation from B tool frame to optical frame
        d_k_i = points_transform( Fi(F_BK) @ F_AK , A_tip)
        d_k.append(d_k_i)
    d_k = np.array(d_k).astype(float)
    d_k = d_k.reshape(-1,3)
    return d_k

In [6]:
def ProjectOnSegment(c,p,q):
# c is the point we want to find corresponding closest point
# p q are the start and the and of a segment

#according to https://ciis.lcsr.jhu.edu/lib/exe/fetch.php?media=courses:455-655:lectures:finding_point-pairs.pdf
# page 7
    c = np.array(c).astype(float)
    p = np.array(p).astype(float)
    q = np.array(q).astype(float)
    #print(np.dot( c-p , q-p ))
    #print(np.dot( q-p ,q-p ))
    Lambda = np.dot( c-p , q-p ) / np.dot( q-p ,q-p )
    Lambda = max(0, min(Lambda, 1))
    c_star = p + Lambda*(q-p)
    return c_star

In [7]:
def FindClosestPoint( a, p, q, r ):
    
    A = np.zeros((3,2),dtype = float )
    
    for i in range(3):
        A[i][0] = q[i] - p[i]
        A[i][1] = r[i] - p[i]
        
    B = a - p
    B = B.reshape(-1,1)
    X = np.linalg.lstsq(A, B, rcond=None)[0]
    
    Lambda = X[0]
    Miu = X[1]

    a = p + Lambda * (q - p) + Miu * (r - p)
    P_closest = np.zeros(3)

    if Lambda >= 0 and Miu>=0 and Lambda+Miu<=1:
        P_closest = a
    elif Lambda < 0:
        P_closest = ProjectOnSegment(a, r, p)
    elif Miu < 0:
        P_closest = ProjectOnSegment(a, p, q)
    elif Lambda + Miu > 1:
        P_closest = ProjectOnSegment(a, q, r)

    return P_closest


In [8]:
def point_to_triangle(s_k_i , j):
    p = mesh_points[mesh_vertices[j][0]] 
    q = mesh_points[mesh_vertices[j][1]]
    r = mesh_points[mesh_vertices[j][2]] 
            
    tmp = FindClosestPoint( s_k_i, p, q, r )
    
    return tmp

In [9]:
def bruteforce_matching(s_k , Np , mesh_points , Ntriangles , mesh_vertices ):
    Ns = s_k.shape[0]
    
    closest_p = []
    for i in range(Ns):
        min_dist = np.finfo(np.float32).max
        P_closest = []
        for j in range(Ntriangles):

            tmp = point_to_triangle(s_k[i] , j)

            dist = np.linalg.norm(s_k[i] - tmp)
            if min_dist > dist:
                min_dist = dist
                P_closest = tmp
        closest_p.append(P_closest)
                
    return closest_p

In [10]:
def get_rectangles(Np , mesh_points , Ntriangles , mesh_vertices):
    
    rectangles = np.zeros( (Ntriangles,7) ,dtype=float)
    
    for i in range(Ntriangles):
        p = mesh_points[mesh_vertices[i][0]] 
        q = mesh_points[mesh_vertices[i][1]]
        r = mesh_points[mesh_vertices[i][2]]
        rectangles[i,6] = i
        for j in range(3):
            rectangles[i,j] = min( p[j],q[j],r[j])
        for j in range(3):
            rectangles[i,j+3] = max( p[j],q[j],r[j])
        #print(rectangle[i,0:3] - rectangle[i,3:6])
        
    return rectangles

In [11]:
class KD_tree():
    def __init__(self, Np , mesh_points , Ntriangles , mesh_vertices , rectangles):
        
        self.Np = Np
        self.mesh_points = mesh_points
        self.mesh_vertices = mesh_vertices
        self.Ntriangles =  Ntriangles
        self.rectangles = rectangles
        self.num = np.zeros(Ntriangles+10).astype(int)
        #self.D = mesh_points.shape[1]
        self.D = 3
        
        self.ls = np.zeros( Ntriangles+10 )
        self.rs = np.zeros( Ntriangles+10 )
        
        #self.points = rectangles
        self.Rectangle = np.zeros( (Ntriangles+10,6) )
        
        self.tree = self.build(rectangles.tolist(), 1 , Ntriangles , depth =0 )
        self.nearest_point = []
        self.nearest_dist = 0
        
    def pushup(self, root):
        
        ls = self.ls[root]
        rs = self.rs[root]
        if(ls!=0):
            for i in range(3):
                self.Rectangle[root,i] = min(self.Rectangle[root,i],self.Rectangle[int(ls),i])
                self.Rectangle[root,i+3] = max(self.Rectangle[root,i+3],self.Rectangle[int(ls),i+3]) 
        if(rs!=0):
            for i in range(3):
                self.Rectangle[root,i] = min(self.Rectangle[root,i],self.Rectangle[int(rs),i])
                self.Rectangle[root,i+3] = max(self.Rectangle[root,i+3],self.Rectangle[int(rs),i+3]) 
    
    def point_to_cube(self, start_point, root):
        dis = np.zeros(3)
        for i in range(self.D):
            if(start_point[i] < self.Rectangle[root,i]):
                dis[i] = self.Rectangle[root,i] - start_point[i]
            if(start_point[i] > self.Rectangle[root,i+3]):
                dis[i] = start_point[i] - self.Rectangle[root,i+3]
        dist = np.linalg.norm(dis)
        return dist
    
    def find(self, start_point , left,right,depth):
        if(left>right):
            return np.finfo(np.float32).max
        middle = ((left + right) // 2) 
        
        dist = self.point_to_cube(start_point , middle)
        if(dist > self.nearest_dist):
            return
        
        tmp = point_to_triangle(start_point , self.num[middle])
        dist = np.linalg.norm(start_point - tmp)
        
        if( dist < self.nearest_dist):
            self.nearest_dist = dist
            self.nearest_point = tmp
        
        self.find( start_point , left , middle-1 ,depth)
        self.find( start_point , middle+1 , right,depth)
        
    def FindClosestPoint(self, start_point ):
        
        self.nearest_dist =  np.finfo(np.float32).max
    
        self.find( start_point , 1 , self.Ntriangles , depth=0 ) 
        
        return self.nearest_point
        
    def build( self, points, left,right,depth ):
        if(left>right):
            return 0
        
        axis = depth % self.D

        points.sort(key=itemgetter(axis))
        middle = ((left + right) // 2) 
        #print("points: ",len(points))
        #print("middle: ",middle)
        self.Rectangle[middle] = np.array(points[ middle - left ][0:6]).astype(float)
        self.num[middle] =  points[middle - left ][6]
        
        self.ls[ middle ] = self.build(points[:middle- left] ,left , middle-1 , depth+1 )
        self.rs[ middle ] = self.build(points[middle-left+1:]   ,middle+1, right , depth+1 )
        self.pushup(middle)
        
        return middle

   

In [12]:
def get_difference( A , B):

    dist = np.zeros(( A.shape[0],1))
    for i in range(A.shape[1]):
        dist[i] = np.linalg.norm( A[i,:] - B[i,:])

    return dist

In [13]:
def kd_matching(s_k , Np , mesh_points , Ntriangles , mesh_vertices ):
    
    rectangles = get_rectangles(Np , mesh_points , Ntriangles , mesh_vertices)

    kdtree = KD_tree(Np , mesh_points , Ntriangles , mesh_vertices , rectangles)
    
    Ns = s_k.shape[0]
    
    closest_p = []
    for i in range(Ns):
        min_dist = np.finfo(np.float32).max
        tmp = kdtree.FindClosestPoint( s_k[i] )
        closest_p.append(tmp)
    return closest_p

In [14]:
def ICP(d_k , Np , mesh_points , Ntriangles , mesh_vertices ,brute_force = False ):
    
    F_reg = np.eye(4)
    
    rectangles = get_rectangles(Np , mesh_points , Ntriangles , mesh_vertices)
    if(brute_force == False):
        kdtree = KD_tree(Np , mesh_points , Ntriangles , mesh_vertices , rectangles)
    
    Ns = d_k.shape[0]
    
    closest_p = []
    iteration = 40
    c_k = []
    old_pts = []
    
    closest_p = []

    
    
    for i in range(iteration):
        s_k_i = points_transform(F_reg , d_k) 
        #c_k = icpm.ICPmatch(s_i, vCoords, vIndices, tree=tree, oldpts=old_pts, usetree=True)
        closest_p = []
        if(brute_force):
            for j in range(Ns):
                min_dist = np.finfo(np.float32).max
                P_closest = []
                for k in range(Ntriangles):

                    tmp = point_to_triangle(s_k_i[j] , k )

                    dist = np.linalg.norm(s_k_i[j] - tmp)
                    if min_dist > dist:
                        min_dist = dist
                        P_closest = tmp
                closest_p.append(P_closest)
       
            
        if(brute_force==False):
            for j in range(Ns):
                min_dist = np.finfo(np.float32).max
                tmp = kdtree.FindClosestPoint( s_k_i[j] )
                closest_p.append(tmp)

            
        c_k = np.array(closest_p)
        # Update guess
        old_pts = c_k

        deltaF_reg = registration_3d( s_k_i , c_k)

        F_regNew = deltaF_reg @ F_reg

        if np.sum(np.abs(F_reg - F_regNew)) < 1e-4:
            return c_k , F_regNew

        F_reg = F_regNew


            
    return c_k , F_reg

In [15]:
data_dir = 'DATA/'
letters = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K']
runtype = ['Debug' ,'Debug' ,'Debug' ,'Debug' ,'Debug' ,'Debug' ,'Unknown','Unknown','Unknown','Unknown']
pa = "PA4"
filename = '/Problem4MeshFile.sur'

In [ ]:
if (True):
    num=3
#for num in range(1):
    data_type = num
    letter = num
    Np,mesh_points,Ntriangles,mesh_vertices = read_mesh(data_dir , '/Problem4MeshFile.sur')
    N_Ap,A_points,A_tip = read_body(data_dir , '/Problem4-BodyA.txt')
    N_Bp,B_points,B_tip = read_body(data_dir , '/Problem4-BodyB.txt')
    
    sample_dir = '/PA4-' + letters[num] + '-' + runtype[num] + '-SampleReadingsTest.txt'
    A_frames , B_frames = read_sample(data_dir, sample_dir, N_Ap , N_Bp)
    
    d_k = get_d_k(A_frames, A_points, A_tip, B_frames, B_points, B_tip)
    F_reg = np.eye(4)
    print("d_k points : ",d_k.shape[0])
    c_k, F_reg = ICP(d_k , Np , mesh_points , Ntriangles , mesh_vertices )
    s_k = points_transform(F_reg , d_k) 
    
    difference = get_difference(s_k , c_k)
    print( '/PA4-' + letters[num] + '-' + runtype[num])
    print(np.sum(difference))
    for i in range(s_k.shape[0]):
        print( np.round(s_k[i],2), ' ' ,np.round(c_k[i],2) )

d_k points :  200
